In [1]:
import sys
sys.path.append("/Users/PRVATE/Documents/tf_transformers/src/")

In [2]:
from transformers import TFRobertaModel
from tf_transformers.models import ROBERTAEncoder

import tensorflow as tf
import json

from tf_transformers.utils import convert_roberta_hf_to_tf_transformers
import os

In [3]:
# Load HF model

# Always do this
tf.keras.backend.clear_session()

model_hf_location = '/Users/PRVATE/HUggingFace_Models/roberta-base'
model_hf = TFRobertaModel.from_pretrained(model_hf_location)

Some layers from the model checkpoint at /Users/PRVATE/HUggingFace_Models/roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at /Users/PRVATE/HUggingFace_Models/roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


In [4]:
# Load tf_transformers model
# Most config we will be providing

# Default configs for the model
model_config_dir = '/Users/PRVATE/Documents/tf_transformers/model_configs/'
model_name = 'roberta_base'
config_location = os.path.join(model_config_dir, model_name, 'config.json')
config = json.load(open(config_location))
# Always do this
tf.keras.backend.clear_session()

# tf_transformers Layer (an extension of Keras Layer)
# This is not Keras model, but extension of keras Layer

model_layer = ROBERTAEncoder(config=config,
                      name='roberta',
                      mask_mode=config['mask_mode'],
                      is_training=False, 
                      use_dropout=False,
                      )
# model_dir = None, because we have not initialized the model with proper variable values
model_tf_transformers = model_layer.get_and_load_model(model_dir=None)

INFO:absl:We are overwriding `is_training` is False to `is_training`                     to True with `use_dropout` is False, no effects on your inference pipeline
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids:0", shape=(None, None), dtype=int32)
INFO:absl:input_mask ---> Tensor("input_mask:0", shape=(None, None), dtype=int32)
INFO:absl:input_type_ids ---> Tensor("input_type_ids:0", shape=(None, None), dtype=int32)
INFO:absl:Initialized Variables
INFO:absl:Inputs -->
INFO:absl:input_ids ---> Tensor("input_ids_1:0", shape=(None, None), dtype=int32)
INFO:absl:input_mask ---> Tensor("input_mask_1:0", shape=(None, None), dtype=int32)
INFO:absl:input_type_ids ---> Tensor("input_type_ids_1:0", shape=(None, None), dtype=int32)


In [5]:
convert_roberta_hf_to_tf_transformers(model_hf, model_tf_transformers, config)

INFO:absl:Deleteing huggingface model for saving memory
INFO:absl:We slice Positional Embeddings from 514 to 512
INFO:absl:Done assigning variables weights . Total 199


In [6]:
# Please have a look at tf_transformers/extra/*.py for reference values

input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])

input_mask = tf.ones_like(input_ids)
input_type_ids = tf.zeros_like(input_ids)

inputs = {'input_ids': input_ids, 
          'input_mask': input_mask, 
          'input_type_ids': input_type_ids}

results_tf_transformers   = model_tf_transformers(inputs)
for k, r in results_tf_transformers.items():
    if isinstance(r, list):
        continue
    print(k, '-->', tf.reduce_sum(r), '-->', r.shape)
    
    
# For Roberta Base 

# cls_output --> tf.Tensor(9.341034, shape=(), dtype=float32) --> (2, 768)
# token_embeddings --> tf.Tensor(149.65, shape=(), dtype=float32) --> (2, 5, 768)
# token_logits --> tf.Tensor(-385563.88, shape=(), dtype=float32) --> (2, 5, 50265)
# last_token_logits --> tf.Tensor(-85481.09, shape=(), dtype=float32) --> (2, 50265)

cls_output --> tf.Tensor(9.341352, shape=(), dtype=float32) --> (2, 768)
token_embeddings --> tf.Tensor(149.64793, shape=(), dtype=float32) --> (2, 5, 768)
token_logits --> tf.Tensor(-385577.8, shape=(), dtype=float32) --> (2, 5, 50265)
last_token_logits --> tf.Tensor(-85478.8, shape=(), dtype=float32) --> (2, 50265)


In [7]:
# Huggingface Model
input_ids  = tf.constant([[1, 9, 10, 11, 23], 
                         [1, 22, 234, 432, 2349]])

input_ids  = tf.constant([[1, 2, 3, 4, 5], 
                         [1, 2, 3, 4, 5]])

input_mask = tf.ones_like(input_ids)
input_type_ids = tf.zeros_like(input_ids)

results_hf = model_hf([input_ids, input_mask, input_type_ids])
for k in results_hf:
    print(tf.reduce_sum(k), '-->', k.shape)

tf.Tensor(144.91971, shape=(), dtype=float32) --> (2, 5, 768)
tf.Tensor(7.443221, shape=(), dtype=float32) --> (2, 768)


In [ ]:
model_tf_transformers.save_checkpoint('/Users/PRVATE/tf_transformers_models/roberta-base')

In [10]:
model_tf_transformers.load_checkpoint('/Users/PRVATE/tf_transformers_models/roberta-base/')
# model_name = 'roberta-base'
# model_loc  = '/Users/PRVATE/tf_transformers_models/'
# checkpoint_dir = os.path.join(model_loc, model_name)
# if os.path.exists(checkpoint_dir):
#     raise FileExistsError()

# # If you want to save the model as checkpoints
# checkpoint = tf.train.Checkpoint(model=model_tf_transformers)
# manager = tf.train.CheckpointManager(
#     checkpoint, directory=checkpoint_dir, max_to_keep=1
# )
# manager.save()
# print("Saved at {}".format(manager.latest_checkpoint))

INFO:absl:Succesful: Model checkpoints matched


In [16]:
model_tf_transformers.save("model_pb", save_format='tf')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: model_pb/assets


INFO:tensorflow:Assets written to: model_pb/assets
